In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./train.csv')
#Codification de la variable cible 
train['type_local'] = train['type_local'].map({'Maison':1, 'Appartement':0})
# Ajouter le logarithm des prix à la base
train['logprix'] = np.log(train['estimation_prix'])

# Suppression de la variable estimation_prix 
train= train.drop('estimation_prix', axis=1)

In [3]:
#Codification de la variable cible 
train['type_local'] = train['type_local'].map({'Maison':1, 'Appartement':0})


In [4]:
# Ajouter le logarithm des prix à la base
train['logprix'] = np.log(train['estimation_prix'])

# Suppression de la variable estimation_prix 
train= train.drop('estimation_prix', axis=1)


In [5]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


from sklearn.linear_model import Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor
from xgboost import XGBRegressor
!pip install lightgbm
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.6 MB/s eta 0:00:00a 0:00:01


In [6]:
TGT = 'logprix'
FE = ['adresse_code_voie','code_postal', 'nom_commune', 'code_commune', 'type_local','surface_reelle_bati', 'surface_terrain', 
      'nombre_pieces_principales','longitude', 'latitude', 'departement', 'year']

In [8]:
x_train, x_test, y_train, y_test=train_test_split(train[FE], train[TGT],stratify=train['type_local'],test_size=0.2, random_state=42)

In [9]:
# variable cible
labels=y_train
# variable d'entraînement
X = x_train
# Normalisation des variables d'entraînement
scalar = MinMaxScaler()
X_scaled = scalar.fit_transform(X)

In [10]:
def cross_validation(model, X, y):
    "Check model with cross validation"
    score = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_percentage_error')
    cross_score = (-score)
    return round(100*np.mean(cross_score), 4)

Après consultation de la litterature on plusieurs test on a retenu les modèles ci-dessous a essayer

In [11]:
#liste des modèles
lasso_model = Lasso()
elastic_model = ElasticNet()
xgb_model = XGBRegressor()
lgbm_model=LGBMRegressor()
mlp_model=MLPRegressor()
rf_model=RandomForestRegressor()

LightGBM

In [15]:
# Verification des modèles avec la cross-validation
cv_lgbm_evl= cross_validation(lgbm_model, X_scaled, labels)
print("MAPE : ",cv_lgbm_evl)

MAPE :  2.0967


Lasso

In [12]:
# Verification des modèles avec la cross-validation
cv_lgbm_evl= cross_validation(lasso_model, X_scaled, labels)
print("MAPE : ",cv_lgbm_evl)

MAPE :  3.7935


ElasticNet

In [13]:
# Verification des modèles avec la cross-validation
cv_lgbm_evl= cross_validation(elastic_model, X_scaled, labels)
print("MAPE : ",cv_lgbm_evl)

MAPE :  3.7935


XGBoost

In [15]:
# Verification des modèles avec la cross-validation
cv_xgb_evl= cross_validation(xgb_model, X_scaled, labels)
print("MAPE : ",cv_xgb_evl)

MAPE :  2.0189


AdaBoost

RandomForest

In [ ]:
# Verification des modèles avec la cross-validation
cv_lgbm_evl= cross_validation(rf_model, X_scaled, labels)
print("MAPE : ",cv_lgbm_evl)

**Observation**

On constate que l'erreur absolue moyen en pourcentage est assez élevé pour chacun de ces derniers modèles.(Les valeurs des sont petites  car on cible le log des prix et non les prix)
Il est apparaît necessaire de combiner la force de plusieurs modèle pour réduire le MAPE qui est la métriqiue ciblé.Par exemple en faisant un stacking ou un voting des meilleures modèles tout en optimisant leurs paramètres.


**DETECTION DES MEILLEURS MODELES ET OPTIMISATION**

On constate que les modèles suivants sont les plus adaptés : **LightGBM, XGBoost, et Randomforest**.

**Optimisation**
Les hyperparametres de ces trois modèles ont été optimer grâce à gridsearch importaé plus haut.



**LGBMRegressor**: colsample_by_tree=0.8, learning_rate=0.12, max_depth=4,min_split_gain=0.3,num_leaves=20,reg_alpha=1.1,subsample=0.8,subsample_freq=20

**RandomForestRegressor**: Les meilleurs paramètres d'après notre espace de reherche ne sont que les parametres par défaut

**XGBRegressor**:learning_rate=0.1, num_leaves=0.3

**Modèle final**

Executons un voting des trois meilleures modèles optimisés

In [ ]:
from sklearn.ensemble import VotingRegressor

voting_reg = VotingRegressor(estimators=[('lgbm', LGBMRegressor(colsample_by_tree=0.8, learning_rate=0.12, max_depth=4,min_split_gain=0.3,num_leaves=20,reg_alpha=1.1,subsample=0.8,subsample_freq=20)), ('rf', RandomForestRegressor()), 
                                         ('bg',XGBRegressor(learning_rate=0.1, num_leaves=0.3))])
voting_reg.fit(x_train, y_train)

predictions_voting_reg = voting_reg.predict(x_test)
MAPE(y_test,predictions_voting_reg)

[LightGBM] [Warning] Unknown parameter: colsample_by_tree


In [ ]:
MAPE(np.exp(y_test),np.exp(predictions_voting_reg))

**Prédiction**

In [ ]:
test = pd.read_csv('./test.csv')
#Codification de la variable cible 
test['type_local'] = test['type_local'].map({'Maison':1, 'Appartement':0})
# Ajouter le logarithm des prix à la base
test['logprix'] = np.log(test['estimation_prix'])
# Suppression de la variable estimation_prix 
test= test.drop('estimation_prix', axis=1)

In [ ]:
pred=voting_reg.predict(test)

In [ ]:
test["estimatipn_prix"] = pred
test.head()

In [ ]:
test[['id', 'prediction']].to_csv("submission.csv", index=False)